### Training


In [ ]:
%load_ext tensorboard

import random
import shutil
import os
import json
import numpy as np

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from sklearn.model_selection import train_test_split


In [ ]:
DATA_PATH = "data.json"

VALIDATION_SPLIT = 0.2 # percentage of dataset
TEST_SPLIT = 0.1 # percentage of dataset

LOGDIR = "logs/hparam_tuning/"
CHECKPOINT_DIR = "logs/checkpoint"

NUM_SESSION_GROUPS = 1
NUM_EPOCHS = 20
BATCH_SIZE = 32

COMPLEX_HPARAM_TUNING = False


In [ ]:
# load training data from json file

with open(DATA_PATH, "r") as f:
    data = json.load(f)

x = np.array(data["mfcc"])
y = np.array(data["labels"])


In [ ]:
# split data into train, validation and test sets

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SPLIT)
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=VALIDATION_SPLIT)

# add an axis to input sets to match the shape CNN expects (last axis is like channel in color images)
x_train = x_train[..., np.newaxis]
x_test = x_test[..., np.newaxis]
x_validation = x_validation[..., np.newaxis]


In [ ]:
# choose hyperparameters to tune 

# some are blocked by an if condition due to machine memory constraints during tuning
if COMPLEX_HPARAM_TUNING == True:
    HP_CONV_LAYERS = hp.HParam("conv_layers", hp.IntInterval(1, 3))
    HP_CONV_KERNEL_SIZE = hp.HParam("conv_kernel_size", hp.Discrete([3, 5]))
    HP_POOL_SIZE = hp.HParam("conv_pool_size", hp.Discrete([2, 3]))

HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([32, 64]))
HP_DENSE_LAYERS = hp.HParam("dense_layers", hp.IntInterval(1, 3))
HP_DROPOUT = hp.HParam("dropout", hp.RealInterval(0.2, 0.3))
HP_OPTIMIZER = hp.HParam("optimizer", hp.Discrete(["adam", "sgd"]))

HPARAMS = [
    HP_NUM_UNITS,
    HP_DENSE_LAYERS,
    HP_DROPOUT,
    HP_OPTIMIZER
]

if COMPLEX_HPARAM_TUNING == True:
    HPARAMS.insert(0, HP_CONV_LAYERS)
    HPARAMS.insert(1, HP_CONV_KERNEL_SIZE)
    HPARAMS.insert(2, HP_POOL_SIZE)

METRICS = [
    hp.Metric(
        "epoch_loss",
        group="validation",
        display_name="loss (val)",
    ),
    hp.Metric(
        "epoch_accuracy",
        group="validation",
        display_name="accuracy (val)",
    )
]


In [14]:
def create_model(hparams, seed):
    rng = random.Random(seed)

    INPUT_SHAPE = (x_train.shape[1], x_train.shape[2], 1)

    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=INPUT_SHAPE))

    if COMPLEX_HPARAM_TUNING == True:
        conv_filters = 8
        for _ in range(hparams[HP_CONV_LAYERS]):
            model.add(
                tf.keras.layers.Conv2D(
                    filters=conv_filters,
                    kernel_size=hparams[HP_CONV_KERNEL_SIZE],
                    padding="same",
                    activation="relu"
                )
            )
            model.add(
                tf.keras.layers.MaxPooling2D(
                    pool_size=hparams[HP_POOL_SIZE],
                    strides=hparams[HP_POOL_SIZE]-1,
                    padding="same"
                )
            )
            model.add(tf.keras.layers.BatchNormalization())
            conv_filters *= 2
    else:
        # 1st conv layer
        model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=INPUT_SHAPE))
        model.add(tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
        model.add(tf.keras.layers.BatchNormalization())

        # 2nd conv layer
        model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
        model.add(tf.keras.layers.BatchNormalization())

        # 3rd conv layer\n",
        model.add(tf.keras.layers.Conv2D(32, (2, 2), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
        model.add(tf.keras.layers.BatchNormalization())
    
    # flatten output and feed it into dense layer
    model.add(tf.keras.layers.Flatten())

    for _ in range(hparams[HP_DENSE_LAYERS]):
        model.add(tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation="relu"))

    model.add(tf.keras.layers.Dropout(hparams[HP_DROPOUT], seed=rng.random()))

    # output layer
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    model.compile(
       optimizer=hparams[HP_OPTIMIZER],
       loss='sparse_categorical_crossentropy',
       metrics=['accuracy']
    )

    return model


In [ ]:
def run(base_logdir, session_id, hparams):
    model = create_model(hparams=hparams, seed=session_id)
    logdir = os.path.join(base_logdir, session_id)
    checkpoint_dir = os.path.join(CHECKPOINT_DIR, session_id)

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=logdir,
        histogram_freq=1
    )

    hparams_callback = hp.KerasCallback(logdir, hparams)

    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_dir,
        save_weights_only=True,
        monitor="val_accuracy",
        mode="max",
        save_best_only=True,
        verbose=1
    )

    model.fit(
        x=x_train[:BATCH_SIZE],
        y=y_train[:BATCH_SIZE],
        epochs=NUM_EPOCHS,
        batch_size=BATCH_SIZE,
        shuffle=False,
        validation_data=(x_validation, y_validation),
        callbacks=[tensorboard_callback, hparams_callback, checkpoint_callback]
    )

    return model


In [ ]:
def run_all(logdir, verbose=True):
    rng = random.Random(0)

    with tf.summary.create_file_writer(logdir).as_default():
        hp.hparams_config(hparams=HPARAMS, metrics=METRICS)

    # randomly select hyperparameters
    sessions_per_group = 2
    num_sessions = NUM_SESSION_GROUPS * sessions_per_group
    session_index = 0  # across all session groups
    for _ in range(NUM_SESSION_GROUPS):
        hparams = {h: h.domain.sample_uniform(rng) for h in HPARAMS}
        hparams_string = str(hparams)
        for repeat_index in range(sessions_per_group):
            session_id = str(session_index)
            session_index += 1
            if verbose:
                print(
                    "--- Running training session %d/%d"
                    % (session_index, num_sessions)
                )
                print(hparams_string)
                print("--- repeat #: %d" % (repeat_index + 1))
            model = run(
                base_logdir=logdir,
                session_id=session_id,
                hparams=hparams,
            )
    
    return model


In [ ]:
# main

np.random.seed(0)
logdir = LOGDIR
shutil.rmtree("./logs/", ignore_errors=True)
print(f"Saving output to {logdir}")
model = run_all(logdir=logdir, verbose=True)
print(f"Done. Output saved to {logdir}")


In [ ]:
%tensorboard --logdir logs/hparam_tuning

In [ ]:
# load best model from checkpoint

# RUN_ID = "0"
# model.load_weights(os.path.join(LOGDIR, RUN_ID))
